In [ ]:
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

from keras.models import Model, Sequentia
from keras.layers import Input, GlobalMaxPooling1D, Convolution1D, Dot
from keras.layers.merge import Concatenate

from keras import regularizers
from attention_layer import *

## Text

In [ ]:
# Simple LSTM

model_LSTM = Sequential()
model_LSTM.add(Embedding(max_features,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))
#model_LSTM.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model_LSTM.add(LSTM(64))
model_LSTM.add(Dropout(0.5))
model_LSTM.add(Dense(1))
model_LSTM.add(Activation('sigmoid'))

model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

model_LSTM.layers[0].trainable = True
model_LSTM.compile(loss="binary_crossentropy", optimizer="adam", metrics = ["accuracy"])

In [ ]:
# Simple GRU

model_GRU = Sequential()
model_GRU.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model_GRU.add(GRU(32, dropout=0.5, recurrent_dropout=0.5))
model_GRU.add(Dense(1))
model_GRU.add(Activation('sigmoid'))

model_GRU.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
# Simple WeightNormGRU

model = Sequential()
model.add(Embedding(max_features,
                    embedding_dim,
                    input_length=maxlen))
model.add(WeightNormGRU(64, return_sequences=True))
model.add(WeightNormGRU(64))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', metrics = ['accuracy'])

In [ ]:
# MultiOutputLSTM with 2 losses

input_layer = Input(shape=(maxlen,))

model = Embedding(max_features, embedding_dim, embeddings_regularizer=regularizers.l1(10e-5))(input_layer)
model = LSTM(64,return_sequences = True)(model)
model = LSTM(64)(model)
model = Dropout(0.5)(model)

out1 = Dense(5, activation = 'softmax',name = 'out1')(model)
out2 = Dense(1, activation = 'relu', name = 'out2')(model)

model = Model(input_layer, outputs=[out1, out2])

model.compile(optimizer='adam',
              loss={'out1': 'categorical_crossentropy', 'out2': 'mae'}, metrics = ['accuracy'])

In [ ]:
# LSTM's (initial state), shared Embeddings

input_title = Input(shape=(maxlen,))
input_review = Input(shape=(maxlen,))

shared_embedding = Embedding(max_features, embedding_dim, embeddings_regularizer = regularizers.l1(10e-5))

title = shared_embedding(input_title)
review = shared_embedding(input_review)

title = LSTM(64, return_state = True)(title)
model = LSTM(64)(review, initial_state = title[1:])

out = Dense(5, activation = 'softmax')(model)
model = Model([input_title,input_review], out)

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics = ['accuracy'])

In [ ]:
# LSTM + Attention

input_layer = Input(shape=(maxlen,))
embeddings = Embedding(max_features, embedding_dim, input_length=maxlen, embeddings_regularizer = regularizers.l1(10e-5))(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embeddings)

attention = AttentionWithContext()(lstm_layer)

model = Dot(axes=0)([lstm_layer,attention])
model = Dropout(0.5)(model)

out = Dense(5, activation='softmax')(model)

model = Model(inputs=input_layer, outputs=out)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics = ['accuracy'])

In [ ]:
# Text CNN

filter_sizes = 7
num_filters = 100
dropout_prob = (0.5, 0.5)
hidden_dims = 32
input_shape = (maxlen,)

model_input = Input(shape=input_shape)

model_CNN = Embedding(max_features, embedding_dim, input_length=maxlen, name="embedding")(model_input)
model_CNN = Dropout(dropout_prob[0])(model_CNN)

model_CNN = Convolution1D(filters=num_filters,kernel_size=filter_sizes,padding="valid",activation="relu",strides=1)(model_CNN)
model_CNN = MaxPooling1D(pool_size=2)(model_CNN)
model_CNN = Flatten()(model_CNN)

model_CNN = Dropout(dropout_prob[1])(model_CNN)
model_CNN = Dense(hidden_dims, activation="relu")(model_CNN)

model_output = Dense(1, activation="sigmoid")(model_CNN)

model_CNN = Model(model_input, model_output)
model_CNN.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Text CNN (different filters)

filter_sizes = (3,5,7)
num_filters = 64
dropout_prob = (0.2, 0.3)
hidden_dims = 32
input_shape = (maxlen,)

model_input = Input(shape=input_shape)

model_CNN = Embedding(max_features,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False)(model_input)

model_CNN = Dropout(dropout_prob[0])(model_CNN)

conv_blocks = []
for sz in filter_sizes:
    conv = Convolution1D(filters=num_filters,
                         kernel_size=sz,
                         padding="valid",
                         activation="relu",
                         strides=1)(model_CNN)
    conv = GlobalMaxPooling1D()(conv)
    conv_blocks.append(conv)

model_CNN = Concatenate()(conv_blocks)

model_CNN = Dropout(dropout_prob[1])(model_CNN)
model_CNN = Dense(50, activation = 'relu')(model_CNN)

model_output = Dense(1, activation="sigmoid",use_bias = False)(model_CNN)

model_CNN = Model(model_input, model_output)
model_CNN.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", auc])